# Results Notebook

The aim of this notebook is to generate plots to showcase experiment results at `results/`

Following is plotted:
- SID plots of
    - Gradual - V1
        - SID score - Random graphs - Baselines, existing ABAPC, 4 methods with V1 improved ABAPC
        - SID score - BNlearn 3 datasets - Baselines, existing ABAPC, 4 methods with V1 improved ABAPC
        - Runtime - Random graphs - Baselines, existing ABAPC, 4 methods with V1 improved ABAPC
    - Gradual - V2 - Ablation
        - SID scores - Random graphs - ablation plots
        - SID scores - Random graphs - comparison to baselines, existing ABAPC, V2 best (least approx)
        - Runtime - Random graphs - ablation plots
        - Runtime - Random graphs - comparison to baselines, existing ABAPC, V2. May be combined with last plot

In [18]:
import pandas as pd
import numpy as np

import importlib
import utils
importlib.reload(utils)
from utils import plot_runtime, double_bar_chart_plotly, process_data, DAG_NODES_MAP, DAG_EDGES_MAP

## V1 Plots

In [19]:
baselines_data_random = pd.read_csv('../../results/existing/random_graphs/all_existing_methods_metrics_cpdag.csv')
baselines_data_random_fgs = pd.read_csv('../../results/existing/random_graphs/fgs/all_existing_methods_metrics_cpdag.csv')
baselines_data_random = pd.concat([baselines_data_random, baselines_data_random_fgs], ignore_index=True)

baselines_data_bn = pd.read_csv('../../results/existing/bnlearn_graphs/all_existing_methods_metrics_cpdag.csv')
baselines_data_bn_fgs = pd.read_csv('../../results/existing/bnlearn_graphs/fgs/all_existing_methods_metrics_cpdag.csv')
baselines_data_bn = pd.concat([baselines_data_bn, baselines_data_bn_fgs], ignore_index=True)

v1_data_random = pd.read_csv('../../results/gradual/v1_random_graphs/cpdag_metrics.csv')
v1_data_bn = pd.read_csv('../../results/gradual/v1_bnlearn_graphs/cpdag_metrics.csv')



In [20]:
v1_data_random['elapsed'] = (
    v1_data_random['elapsed_bsaf_creation'] +
    v1_data_random['elapsed_model_solution'] +
    v1_data_random['aba_elapsed'] +
    v1_data_random['ranking_elapsed']
)
v1_data_random['dataset'] = '|V|=' + v1_data_random['n_nodes'].astype(str) + ', |E|=' + v1_data_random['n_edges'].astype(str)
baselines_data_random['dataset'] = '|V|=' + baselines_data_random['n_nodes'].astype(str) + ', |E|=' + baselines_data_random['n_edges'].astype(str)

In [21]:
v1_data_random['model_raw'] = v1_data_random['fact_ranking_method'] + ' ' + v1_data_random['model_ranking_method']
v1_data_random = v1_data_random[v1_data_random['model_raw'].isin([
    'original original',
    'refined_indep_facts original',
    'original refined_indep_facts',
    'original arrows_sum', 
    'original arrows_mean'
])].copy()
v1_data_random['model'] = v1_data_random['model_raw'].map({
    'original original': 'ABAPC (Original)',
    'refined_indep_facts original': 'V1.1 Refined Fact Ranking',
    'original refined_indep_facts': 'V1.2 Model Selection by Refined Fact Strengths',
    'original arrows_sum': 'V1.3 Model Selection by Arrows Sum',
    'original arrows_mean': 'V1.4 Model Selection by Arrows Mean'
})


In [22]:
baselines_random_processed = process_data(baselines_data_random)
v1_random_processed = process_data(v1_data_random)

v1_random_plot_data = pd.concat([v1_random_processed, 
                                 baselines_random_processed[baselines_random_processed['n_nodes'] <= 6]], 
                                 ignore_index=True)

In [23]:
v1_random_plot_data.head(1)

,dataset,n_nodes,n_edges,model,sid_low_mean,sid_high_mean,sid_low_std,sid_high_std,p_SID_low_mean,p_SID_high_mean,p_SID_low_std,p_SID_high_std
0,"|V|=3, |E|=3",3,3,ABAPC (Original),1.66,5.32,2.105484,1.316148,0.553333,1.773333,0.701828,0.438716


In [24]:
methods = [
    'Random',
    'FGS',
    'NOTEARS-MLP',
    'MPC',
    'ABAPC (Original)',
    'V1.1 Refined Fact Ranking',
    'V1.2 Model Selection by Refined Fact Strengths',
    'V1.3 Model Selection by Arrows Sum',
    'V1.4 Model Selection by Arrows Mean'
]
names_dict = {
    'Random': 'Random',
    'FGS': 'FGS',
    'NOTEARS-MLP': 'NOTEARS-MLP',
    'MPC': 'MPC',
    'ABAPC (Original)': 'ABAPC (Original)',
    'V1.1 Refined Fact Ranking': 'V1.1',
    'V1.2 Model Selection by Refined Fact Strengths': 'V1.2',
    'V1.3 Model Selection by Arrows Sum': 'V1.3',
    'V1.4 Model Selection by Arrows Mean': 'V1.4 ',
}
colors_dict = {
        'Random': 'grey',
        'FGS': '#b85c00',
        'NOTEARS-MLP': '#9454c4',
        'MPC': '#379f9f',
        'ABAPC (Original)': '#0085CA',
        'V1.1 Refined Fact Ranking': "#9F8D00",
        'V1.2 Model Selection by Refined Fact Strengths': "#9E0000",
        'V1.3 Model Selection by Arrows Sum': "#ff34da",
        'V1.4 Model Selection by Arrows Mean': "#008100"
    }

In [25]:
import importlib
import utils
importlib.reload(utils)
from utils import double_bar_chart_plotly

double_bar_chart_plotly(v1_random_plot_data, 
                        names_dict,
                        colors_dict,
                        methods=methods,
                        dist_between_lines=0.1175,
                        intra_dis=0.112,
                        inter_dis=0.137,
                        lin_space=5,
                        nl_space=5,
                        start_pos = 0.03,
                            width=1300,
                            height=600,
                            range_y1=(0, 5),
                            range_y2=(0, 5))

Now bnlearn datasets for V1

In [26]:
v1_data_bn['n_nodes'] = v1_data_bn['dataset'].map(DAG_NODES_MAP)
v1_data_bn['n_edges'] = v1_data_bn['dataset'].map(DAG_EDGES_MAP)
baselines_data_bn['n_nodes'] = baselines_data_bn['dataset'].map(DAG_NODES_MAP)
baselines_data_bn['n_edges'] = baselines_data_bn['dataset'].map(DAG_EDGES_MAP)

v1_data_bn['dataset'] =  v1_data_bn['dataset'] + ' <br> ' + '|V|=' + v1_data_bn['n_nodes'].astype(str) + ', |E|=' + v1_data_bn['n_edges'].astype(str)
baselines_data_bn['dataset'] = baselines_data_bn['dataset'] + ' <br> ' + '|V|=' + baselines_data_bn['n_nodes'].astype(str) + ', |E|=' + baselines_data_bn['n_edges'].astype(str)
v1_data_bn['elapsed'] = (
    v1_data_bn['elapsed_bsaf_creation'] +
    v1_data_bn['elapsed_model_solution'] +
    v1_data_bn['aba_elapsed'] +
    v1_data_bn['ranking_elapsed']
)
v1_data_bn['model_raw'] = v1_data_bn['fact_ranking_method'] + ' ' + v1_data_bn['model_ranking_method']
v1_data_bn = v1_data_bn[v1_data_bn['model_raw'].isin([
    'original original',
    'refined_indep_facts original',
    'original refined_indep_facts',
    'original arrows_sum', 
    'original arrows_mean'
])].copy()
v1_data_bn['model'] = v1_data_bn['model_raw'].map({
    'original original': 'ABAPC (Original)',
    'refined_indep_facts original': 'V1.1 Refined Fact Ranking',
    'original refined_indep_facts': 'V1.2 Model Selection by Refined Fact Strengths',
    'original arrows_sum': 'V1.3 Model Selection by Arrows Sum',
    'original arrows_mean': 'V1.4 Model Selection by Arrows Mean'
})

baselines_bn_processed = process_data(baselines_data_bn)
v1_bn_processed = process_data(v1_data_bn)

v1_bn_plot_data = pd.concat([v1_bn_processed, 
                                 baselines_bn_processed[baselines_bn_processed['n_nodes'] <= 6]], 
                                 ignore_index=True)
# v1_bn_plot_data.loc[v1_bn_plot_data['model'] == 'ABAPC (Ours)', 'model'] = 'ABAPC (Original)'

In [27]:
v1_bn_plot_data.head(1)

,dataset,n_nodes,n_edges,model,sid_low_mean,sid_high_mean,sid_low_std,sid_high_std,p_SID_low_mean,p_SID_high_mean,p_SID_low_std,p_SID_high_std
0,"cancer <br> |V|=5, |E|=4",5,4,ABAPC (Original),8.62,11.3,2.783992,2.101991,2.155,2.825,0.695998,0.525498


In [28]:
double_bar_chart_plotly(v1_bn_plot_data, 
                        names_dict,
                        colors_dict,
                        methods=methods,
                        dist_between_lines=0.1565,
                        lin_space=6,
                        nl_space=6,
                        intra_dis = 0.161,
                        inter_dis = 0.174,
                        start_pos = 0.04,
                            width=1300,
                            height=600,
                            range_y1=(0, 5),
                            range_y2=(0, 5))

Runtime for V1

In [32]:
methods = [
    # 'Random',
    # 'FGS',
    # 'NOTEARS-MLP',
    # 'MPC',
    'ABAPC (Original)',
    'V1.1 Refined Fact Ranking',
    'V1.2 Model Selection by Refined Fact Strengths',
    'V1.3 Model Selection by Arrows Sum',
    'V1.4 Model Selection by Arrows Mean'
]

symbols_dict = {
    'ABAPC (Original)':'triangle-down-dot',
    'FGS':'triangle-up-dot',
    'NOTEARS-MLP':'pentagon-dot',
    'MPC':'hexagon2-dot',
    'Random':'x', 
    'V1.1 Refined Fact Ranking':'circle-dot',
    'V1.2 Model Selection by Refined Fact Strengths':'diamond-dot',
    'V1.3 Model Selection by Arrows Sum':'square-dot',
    'V1.4 Model Selection by Arrows Mean':'star-dot',}

In [35]:
v1_data_for_runtime_plot = pd.concat([v1_data_random, baselines_data_random], ignore_index=True)
v1_data_for_runtime_plot.loc[v1_data_for_runtime_plot['model'] == 'ABAPC (Original)', 'elapsed'] = (
    v1_data_for_runtime_plot.loc[v1_data_for_runtime_plot['model'] == 'ABAPC (Original)', 'aba_elapsed'] + 
    v1_data_for_runtime_plot.loc[v1_data_for_runtime_plot['model'] == 'ABAPC (Original)', 'ranking_elapsed'])
v1_data_for_runtime_plot = v1_data_for_runtime_plot.groupby(['n_nodes', 'n_edges', 'model'], as_index=False).aggregate(
    elapsed_mean=('elapsed', 'mean'),
    elapsed_std=('elapsed', 'std')
)

In [36]:
v1_data_for_runtime_plot = v1_data_for_runtime_plot[v1_data_for_runtime_plot['n_nodes'] <= 6]
plot_runtime(v1_data_for_runtime_plot,
             colors_dict=colors_dict,
             names_dict=names_dict,
             symbols_dict=symbols_dict,
             methods=methods)